In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')
user_data = pd.read_csv('./12345.csv')    # 유저 데이터셋 예시

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 사용자 키워드
# 각 사용자의 키워드를 딕셔너리로 조회
# 아래는 예시 데이터

# 사용자 정보를 담을 빈 딕셔너리 생성
users = {}

# 데이터프레임 순회
for index, row in user_data.iterrows():
    # 각 유저의 정보 추출
    user_name = row['이름']
    user_height = row['키']
    user_weight = row['몸무게']
    user_gender = row['성별']
    user_keywords = row['키워드']

    # BMI 계산
    height_m = user_height / 100
    weight_kg = user_weight
    bmi = weight_kg / (height_m ** 2)

    # 추정된 칼로리 요구량 계산
    if user_gender == '남':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * 33)
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * 33)

    calories_threshold = bmr * 1.2

    # 사용자 정보를 딕셔너리에 추가
    users[user_name] = {'Keywords': user_keywords, 'Calories_threshold': calories_threshold}

# 추천 함수 정의
def get_recommendations(user_keywords, tfidf_matrix, food_data, calories_threshold):
    # 사용자 키워드를 조회할떼 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf.transform([' '.join(user_keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 반환
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores = sim_scores[1:]

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    recommended_food = food_data.iloc[food_indices][['Name','Calories']]

    # 추천된 음식 중에서 사용자가 섭취 가능한 일일 칼로리 이하의 음식만 필터링
    filtered_recommendations = recommended_food[recommended_food['Calories'] <= calories_threshold]
    
    return filtered_recommendations

# 결과 출력
for user_name, user_info in users.items():
    print(user_name, "님의 필터된 음식 추천:")
    print("추정된 칼로리 요구량(1끼):", (user_info['Calories_threshold'])/3)
    recommendations = get_recommendations(user_info['Keywords'], tfidf_matrix, food, user_info['Calories_threshold'])
    print(recommendations)

홍길동 님의 필터된 음식 추천:
추정된 칼로리 요구량(1끼): 697.2405600000001
                                                    Name  Calories
1                                                Biryani    1110.7
2                                          Best Lemonade     311.1
3                         Carina's Tofu-Vegetable Kebabs     536.1
4                                           Cabbage Soup     103.6
5                                 Warm Chicken A La King     895.5
...                                                  ...       ...
165892                  Minty Whipped Shortbread Cookies      62.5
165893                     Spanish Coffee with Tia Maria      84.3
165894                         Meg's Pumpkin Spice Bread     898.2
165895  Roast Prime Rib au Poivre with Mixed Peppercorns    2063.4
165896          Quick & Easy Asian Cucumber Salmon Rolls      16.1

[162224 rows x 2 columns]
이순신 님의 필터된 음식 추천:
추정된 칼로리 요구량(1끼): 690.6902
                                                    Name  Calories
1    